In [1]:
import sys
!{sys.executable} -m pip install pyspark
!{sys.executable} -m pip install findspark


In [2]:
import findspark
findspark.init("C:\spark\spark-3.3.0-bin-hadoop3")
findspark.find()

'C:\\spark\\spark-3.3.0-bin-hadoop3'

In [3]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("covid") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local appName=covid>

In [4]:
import os
def read_cvs_past_top_parquet(path):
        for filename in os.listdir(path):
            
            df = spark.read.csv(path+'/'+filename, header = True)
            print(path+'/'+filename+'/')
            f=filename.split('.')[0]
            #print(filename.split('.')[0])
            df.repartition(2).write.mode('overwrite').parquet('tmp/'+f+'/')
        #df.show()
        

In [5]:
read_cvs_past_top_parquet('archive')



archive/country_wise_latest.csv/
archive/covid_19_clean_complete.csv/
archive/day_wise.csv/
archive/full_grouped.csv/
archive/usa_county_wise.csv/
archive/worldometer_data.csv/


In [6]:
def read_parquet_past_to_pandas(path):
      
            parqDF=spark.read.parquet('tmp/'+path+'/*.parquet')
            #parqDF.show()
            
            df=parqDF.pandas_api()
            return df
            
            #print(parqPandas_clean_complete.head())

In [7]:
parqPandas_country_wise_latest=read_parquet_past_to_pandas('country_wise_latest')
parqPandas_covid_19_clean_complete=read_parquet_past_to_pandas('covid_19_clean_complete')
parqPandas_day_wise=read_parquet_past_to_pandas('day_wise')
parqPandas_full_grouped=read_parquet_past_to_pandas('full_grouped')
parqPandas_usa_country_wise=read_parquet_past_to_pandas('usa_county_wise')
parqPandas_worldometer_data=read_parquet_past_to_pandas('worldometer_data')

In [8]:
parqPandas_covid_19_clean_complete.head()
parqPandas_covid_19_clean_complete.rename(columns={"Date":"date","Province/State":"state","Country/Region":"country","Lat":"lat",
                  "Long":"long","Confirmed":"confirmed","Deaths":"deaths","Recovered":"recovered"},inplace=True)


In [9]:
parqPandas_covid_19_clean_complete['state'] = parqPandas_covid_19_clean_complete['state'].fillna('')
parqPandas_covid_19_clean_complete.isna().sum()

state         0
country       0
lat           0
long          0
date          0
confirmed     0
deaths        0
recovered     0
Active        0
WHO Region    0
dtype: int64

In [10]:
parqPandas_covid_19_clean_complete.deaths=parqPandas_covid_19_clean_complete.deaths.astype(int)
parqPandas_covid_19_clean_complete.confirmed=parqPandas_covid_19_clean_complete.confirmed.astype(int)

In [11]:
confirmed_cases = parqPandas_covid_19_clean_complete.groupby("country")["confirmed"].sum().reset_index()
confirmed_cases

,country,confirmed
0,Chad,64226
1,Russia,45408411
2,Paraguay,156373
3,Yemen,67180
4,Senegal,467457
5,Cabo Verde,82732
6,Sweden,4973160
7,Guyana,19089
8,Eritrea,11786
9,Burma,25188


In [12]:
grouped = parqPandas_covid_19_clean_complete.groupby('country')[['deaths', 'confirmed']].sum().reset_index().sort_values('deaths', ascending=False)[:20]

grouped.deaths=grouped.deaths.astype(int)
grouped.confirmed=grouped.confirmed.astype(int)
grouped

,country,deaths,confirmed
82,US,11011411,224345948
182,United Kingdom,3997775,26748587
143,Brazil,3938034,89524967
68,Italy,3707717,26745145
27,France,3048524,21210926
72,Spain,3033030,27404045
103,Mexico,1728277,14946202
52,India,1111831,40883464
83,Iran,1024136,19339267
38,Belgium,963679,6281116


In [13]:
parqPandas_day_wise.head()

,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,2020-06-20,8805336,465222,4298603,4041511,157552,4249,115305,5.28,48.82,10.82,187
1,2020-04-28,3108149,221974,884680,2001495,75404,6463,32298,7.14,28.46,25.09,184
2,2020-02-17,72815,1868,11951,58996,1936,98,1632,2.57,16.41,15.63,27
3,2020-01-29,6166,133,125,5908,588,2,18,2.16,2.03,106.4,18
4,2020-01-31,9927,213,219,9495,1693,42,78,2.15,2.21,97.26,24


In [14]:
#parqPandas_day_wise=parqPandas_day_wise.rename(columns={"Confirmed":"confirmed"})
parqPandas_day_wise.rename(columns={"Date":"date","Province/State":"state","Country/Region":"country","Lat":"lat",
                  "Long":"long","Confirmed":"confirmed","Deaths":"deaths","Recovered":"recovered"},inplace=True)


In [15]:
parqPandas_day_wise.isna().sum()


date                      0
confirmed                 0
deaths                    0
recovered                 0
Active                    0
New cases                 0
New deaths                0
New recovered             0
Deaths / 100 Cases        0
Recovered / 100 Cases     0
Deaths / 100 Recovered    0
No. of countries          0
dtype: int64

In [16]:
parqPandas_day_wise['New cases']=parqPandas_day_wise['New cases'].astype(int)
parqPandas_day_wise['New deaths']=parqPandas_day_wise['New deaths'].astype(int)

In [17]:
#grouped=grouped.to_pandas
grouped = parqPandas_day_wise.groupby('date')[['New cases', 'New deaths']].sum().reset_index().sort_values('New cases', ascending=False)[:20]
grouped

,date,New cases,New deaths
109,2020-07-23,282756,9966
89,2020-07-24,281164,6144
106,2020-07-22,280647,6983
9,2020-07-25,255545,4867
132,2020-07-16,252544,5799
38,2020-07-17,242038,6743
142,2020-07-18,237635,5627
155,2020-07-21,233565,6238
163,2020-07-10,232577,5311
179,2020-07-15,231122,5493


In [18]:
type(parqPandas_covid_19_clean_complete)

pyspark.pandas.frame.DataFrame

In [19]:
type(parqPandas_day_wise)

pyspark.pandas.frame.DataFrame

In [45]:
df=parqPandas_covid_19_clean_complete.set_index(['date','Active','confirmed', 'deaths', 'recovered']).join(parqPandas_day_wise.set_index(['date','Active','confirmed', 'deaths', 'recovered']))
df.head(100)

,,,,,state,country,lat,long,WHO Region,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
date,Active,confirmed,deaths,recovered,,,,,,,,,,,,
2020-06-07,1132,17039,917,14990,,Japan,36.204824,138.252924,Western Pacific,NaN,NaN,None,None,None,None,None
2020-04-04,139,585,8,438,Beijing,China,40.1824,116.4142,Western Pacific,NaN,NaN,None,None,None,None,None
2020-06-27,277,1531,21,1233,,Zambia,-13.133897,27.849332,Africa,NaN,NaN,None,None,None,None,None
2020-04-03,6,6,0,0,Yukon,Canada,64.2823,-135.0,Americas,NaN,NaN,None,None,None,None,None
2020-03-21,3,3,0,0,,Liberia,6.4280550000000005,-9.429499,Africa,NaN,NaN,None,None,None,None,None
2020-05-15,31,140,1,108,Jilin,China,43.6661,126.1923,Western Pacific,NaN,NaN,None,None,None,None,None
2020-02-05,0,0,0,0,,Grenada,12.1165,-61.678999999999995,Americas,NaN,NaN,None,None,None,None,None
2020-05-31,113,370,1,256,,Rwanda,-1.9403,29.8739,Africa,NaN,NaN,None,None,None,None,None
2020-03-23,659,669,6,4,New South Wales,Australia,-33.8688,151.2093,Western Pacific,NaN,NaN,None,None,None,None,None


In [46]:
parqPandas_covid_19_clean_complete.join(parqPandas_day_wise, lsuffix='_left', rsuffix='_right')

,state,country,lat,long,date_left,confirmed_left,deaths_left,recovered_left,Active_left,WHO Region,date_right,confirmed_right,deaths_right,recovered_right,Active_right,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,,Japan,36.204824,138.252924,2020-06-07,17039,917,14990,1132,Western Pacific,2020-06-20,8805336,465222,4298603,4041511,157552.0,4249.0,115305,5.28,48.82,10.82,187
1,Beijing,China,40.1824,116.4142,2020-04-04,585,8,438,139,Western Pacific,2020-04-28,3108149,221974,884680,2001495,75404.0,6463.0,32298,7.14,28.46,25.09,184
2,,Zambia,-13.133897,27.849332,2020-06-27,1531,21,1233,277,Africa,2020-02-17,72815,1868,11951,58996,1936.0,98.0,1632,2.57,16.41,15.63,27
3,Yukon,Canada,64.2823,-135.0,2020-04-03,6,0,0,6,Americas,2020-01-29,6166,133,125,5908,588.0,2.0,18,2.16,2.03,106.4,18
4,,Liberia,6.4280550000000005,-9.429499,2020-03-21,3,0,0,3,Africa,2020-01-31,9927,213,219,9495,1693.0,42.0,78,2.15,2.21,97.26,24
5,Jilin,China,43.6661,126.1923,2020-05-15,140,1,108,31,Western Pacific,2020-04-25,2891199,206979,798239,1885981,84932.0,5578.0,26910,7.16,27.61,25.93,184
6,,Grenada,12.1165,-61.678999999999995,2020-02-05,0,0,0,0,Americas,2020-07-05,11454847,534150,6105546,4815151,182695.0,3445.0,119171,4.66,53.3,8.75,187
7,,Rwanda,-1.9403,29.8739,2020-05-31,370,1,256,113,Africa,2020-07-18,14292198,602130,7944550,5745518,237635.0,5627.0,150790,4.21,55.59,7.58,187
8,New South Wales,Australia,-33.8688,151.2093,2020-03-23,669,6,4,659,Western Pacific,2020-06-24,9408254,483328,4677005,4247921,171183.0,5261.0,115309,5.14,49.71,10.33,187
9,,Armenia,40.0691,45.0382,2020-05-04,2507,39,1071,1397,Europe,2020-03-24,424889,19016,105997,299876,41154.0,2268.0,9541,4.48,24.95,17.94,171


In [23]:
parqPandas_covid_19_clean_complete.merge(parqPandas_day_wise, on=['date','confirmed','deaths','recovered','Active'] ,how='outer')


,state,country,lat,long,date,confirmed,deaths,recovered,Active,WHO Region,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,,Guinea-Bissau,11.8037,-15.1804,2020-01-22,0,0,0,0,Africa,NaN,NaN,None,None,None,None,None
1,,Luxembourg,49.8153,6.1296,2020-01-22,0,0,0,0,Europe,NaN,NaN,None,None,None,None,None
2,Guadeloupe,France,16.265,-61.551,2020-01-22,0,0,0,0,Europe,NaN,NaN,None,None,None,None,None
3,Bermuda,United Kingdom,32.3078,-64.7505,2020-01-22,0,0,0,0,Europe,NaN,NaN,None,None,None,None,None
4,,Suriname,3.9193,-56.0278,2020-01-22,0,0,0,0,Americas,NaN,NaN,None,None,None,None,None
5,,Zimbabwe,-19.015438,29.154857,2020-01-22,0,0,0,0,Africa,NaN,NaN,None,None,None,None,None
6,Aruba,Netherlands,12.5211,-69.9683,2020-01-22,0,0,0,0,Europe,NaN,NaN,None,None,None,None,None
7,,India,20.593684,78.96288,2020-01-22,0,0,0,0,South-East Asia,NaN,NaN,None,None,None,None,None
8,,Belize,17.1899,-88.4976,2020-01-22,0,0,0,0,Americas,NaN,NaN,None,None,None,None,None
9,,Singapore,1.2833,103.8333,2020-01-22,0,0,0,0,Western Pacific,NaN,NaN,None,None,None,None,None
